# Customer Segmentation Project
by: *Dorothy Kunth*

### Notebook 1 - Data Exploration and Data Preprocessing

## Project Overview
The goal is to carry out segmentation of customers using the transaction history of an online gift store occurring between 01/12/2010 and 09/12/2011.

## Project Scope
1. **Data Exploration**
2. **Data Preprocessing**
3. Exploratory Data Analysis
4. Customer RFM Segmentation using Percentile Ranking
5. Customer RFM Segmentation using Kmeans Clustering

### Connect to Redshift and preview the data

In [1]:
# Import libraries and packages

import os
import sys
import psycopg2
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Add the module path to access functions from src folder
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.extract import connect_to_redshift

# Read connection variables
from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import environment variables from .env file

dbname = os.getenv("dbname")
host = os.getenv("host")
port = os.getenv("port")
user = os.getenv("user")
password = os.getenv("password")

In [3]:
# Connect python to redshift

connect = connect_to_redshift(dbname, host, port, user, password)

Connection to redshift made


In [4]:
# Load and preview online_transactions_cleaned data

query = """
SELECT *
FROM bootcamp.online_transactions_cleaned
ORDER BY customer_id
"""
online_trans = pd.read_sql(query, connect)
online_trans.head()

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,1.04,-74215,-77183.6,2011-01-18 10:17:00,u12346,United Kingdom
1,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,1.04,74215,77183.6,2011-01-18 10:01:00,u12346,United Kingdom
2,581180,23497,CLASSIC CHROME BICYCLE BELL,1.45,12,17.4,2011-12-07 15:52:00,u12347,Iceland
3,581180,21265,PINK GOOSE FEATHER TREE 60CM,1.95,12,23.4,2011-12-07 15:52:00,u12347,Iceland
4,537626,22773,GREEN DRAWER KNOB ACRYLIC EDWARDIAN,1.25,12,15.0,2010-12-07 14:57:00,u12347,Iceland


### Explore the data
- Explore the data and answer business questions using SQL queries

In [5]:
# How many rows are there in the data?

query = """
SELECT COUNT(*)
FROM bootcamp.online_transactions_cleaned;
"""

pd.read_sql(query, connect)

,count
0,399841


In [6]:
# When was the first and last date of order?

query = """
SELECT 
    MIN(invoice_date) as first_invoice_date,
    MAX(invoice_date) as last_invoice_date,
    MAX(invoice_date) - MIN(invoice_date) as num_days
FROM bootcamp.online_transactions_cleaned
"""

pd.read_sql(query, connect)

,first_invoice_date,last_invoice_date,num_days
0,2010-12-01 08:26:00,2011-12-09 12:50:00,373 days 04:24:00


In [7]:
# How many customers does the data contain?

query = """
SELECT COUNT(DISTINCT customer_id) as total_num_customers
FROM bootcamp.online_transactions_cleaned
"""

pd.read_sql(query, connect)

,total_num_customers
0,4363


In [8]:
# How many invoices does the data contain?

query = """
SELECT COUNT(DISTINCT invoice) as total_num_invoices
FROM bootcamp.online_transactions_cleaned
"""

pd.read_sql(query, connect)

,total_num_invoices
0,21791


> Note: There are 399,841 rows and 21,791 invoices which means an invoice could have multiple lines or records.

In [9]:
# How many stock items does the data contain?

query = """
SELECT COUNT(DISTINCT stock_code) as total_num_stocks
FROM bootcamp.online_transactions_cleaned
"""

pd.read_sql(query, connect)

,total_num_stocks
0,3679


In [10]:
# Who were the top 10 performing customers based on total revenue

query = """
SELECT 
    customer_id,
    SUM(total_order_value) as total_revenue
FROM bootcamp.online_transactions_cleaned
GROUP BY customer_id
ORDER BY total_revenue DESC
LIMIT 10
"""

pd.read_sql(query, connect)

,customer_id,total_revenue
0,u14646,278778.02
1,u18102,259657.30
2,u1745,189575.53
3,u14911,132893.24
4,u12415,123638.18
5,u14156,114335.77
6,u17511,88138.20
7,u16684,65920.12
8,u14096,65164.79
9,u13694,62961.54


In [11]:
# Who were the top 10 performing customers based on average revenue

query = """
SELECT 
    customer_id,
    ROUND(SUM(total_order_value)/COUNT(DISTINCT invoice), 2) as avg_revenue
FROM bootcamp.online_transactions_cleaned
GROUP BY customer_id
ORDER BY avg_revenue DESC
LIMIT 10
"""

pd.read_sql(query, connect)

,customer_id,avg_revenue
0,u15098,9904.88
1,u12357,6207.67
2,u15749,5383.98
3,u12415,5151.59
4,u12688,4873.81
5,u1259,4669.19
6,u12752,4366.78
7,u18102,4327.62
8,u18251,4314.72
9,u16,4131.23


In [12]:
# What were the top 10 performing stocks based on revenue

query = """
SELECT 
    stock_code,
    description,
    SUM(total_order_value) as total_revenue
FROM bootcamp.online_transactions_cleaned
GROUP BY stock_code, description
ORDER BY total_revenue DESC
LIMIT 10
"""

pd.read_sql(query, connect)

,stock_code,description,total_revenue
0,22423,REGENCY CAKESTAND 3 TIER,132567.70
1,85123A,CREAM HANGING HEART T-LIGHT HOLDER,93923.15
2,85099B,JUMBO BAG RED RETROSPOT,83056.52
3,47566,PARTY BUNTING,67628.43
4,84879,ASSORTED COLOUR BIRD ORNAMENT,56331.91
5,23084,RABBIT NIGHT LIGHT,51042.84
6,22502,PICNIC BASKET WICKER SMALL,46963.10
7,79321,CHILLI LIGHTS,45915.41
8,22086,PAPER CHAIN KIT 50'S CHRISTMAS,41423.78
9,21137,BLACK RECORD COVER FRAME,38990.63


In [13]:
# What were the top 10 performing stocks based on quantity sold

query = """
SELECT 
    stock_code,
    description,
    SUM(quantity) as total_qty_sold
FROM bootcamp.online_transactions_cleaned
GROUP BY stock_code, description
ORDER BY total_qty_sold DESC
LIMIT 10
"""

pd.read_sql(query, connect)

,stock_code,description,total_qty_sold
0,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,53119
1,22197,POPCORN HOLDER,48689
2,85099B,JUMBO BAG RED RETROSPOT,44963
3,84879,ASSORTED COLOUR BIRD ORNAMENT,35215
4,85123A,CREAM HANGING HEART T-LIGHT HOLDER,34185
5,21212,PACK OF 72 RETROSPOT CAKE CASES,33386
6,23084,RABBIT NIGHT LIGHT,27045
7,22492,MINI PAINT SET VINTAGE,25880
8,22616,PACK OF 12 LONDON TISSUES,25305
9,21977,PACK OF 60 PINK PAISLEY CAKE CASES,24129


In [14]:
# What were the top performing stocks per country based on quantity sold?

query = """
SELECT 
    country, 
    stock_code, 
    qty_sold
FROM (
    SELECT 
        country, 
        stock_code, 
        SUM(quantity) as qty_sold,
        RANK() OVER(PARTITION BY country ORDER BY SUM(quantity) DESC) as rank
    FROM bootcamp.online_transactions_cleaned
    GROUP BY country, stock_code) as sold_per_country
WHERE rank = 1
ORDER BY qty_sold DESC, country;
"""

pd.read_sql(query, connect)

,country,stock_code,qty_sold
0,United Kingdom,84077,47886
1,Netherlands,23084,4801
2,France,23084,3999
3,Japan,23084,3401
4,Australia,22492,2916
...,...,...,...
57,Saudi Arabia,22556,12
58,Saudi Arabia,22969,12
59,Saudi Arabia,22553,12
60,Saudi Arabia,22915,12


In [15]:
# How many orders were placed per month?

query = """
SELECT
    CAST(DATE_PART(YEAR, invoice_date) as int) as year,
    CAST(DATE_PART(MONTH, invoice_date) as int) as month,
    COUNT(DISTINCT invoice) as num_invoices
FROM bootcamp.online_transactions_cleaned
GROUP BY year, month
ORDER BY year, month
"""
pd.read_sql(query, connect)

,year,month,num_invoices
0,2010,12,1692
1,2011,1,1225
2,2011,2,1181
3,2011,3,1588
4,2011,4,1358
5,2011,5,1808
6,2011,6,1686
7,2011,7,1555
8,2011,8,1506
9,2011,9,2038


> There was huge drop in December 2011 since there were only 9 days of transactions.

In [16]:
# How much were the total revenues per month?

query = """
SELECT
    CAST(DATE_PART(YEAR, invoice_date) as int) as year,
    CAST(DATE_PART(MONTH, invoice_date) as int) as month,
    ROUND(SUM(total_order_value), 2) as total_revenue
FROM bootcamp.online_transactions_cleaned
GROUP BY year, month
ORDER BY year, month
"""
pd.read_sql(query, connect)

,year,month,total_revenue
0,2010,12,548443.92
1,2011,1,471580.34
2,2011,2,434218.17
3,2011,3,573838.05
4,2011,4,421527.77
5,2011,5,650735.39
6,2011,6,641129.21
7,2011,7,580714.73
8,2011,8,612966.30
9,2011,9,924390.57


In [17]:
# Did monthly revenues increase/decrease as compared to the previous month?

query = """
SELECT
    year,
    month,
    total_revenue,
    LAG(total_revenue) OVER (ORDER BY month) as prev_month_revenue,
    total_revenue - LAG(total_revenue) OVER (ORDER BY month) as increase_decrease
FROM (
    SELECT
        CAST(DATE_PART(YEAR, invoice_date) as int) as year,
        CAST(DATE_PART(MONTH, invoice_date) as int) as month,
        ROUND(SUM(total_order_value), 2) as total_revenue
    FROM bootcamp.online_transactions_cleaned
    GROUP BY year, month) rev_table
WHERE year = 2011
ORDER BY month;
"""
pd.read_sql(query, connect)

,year,month,total_revenue,prev_month_revenue,increase_decrease
0,2011,1,471580.34,NaN,NaN
1,2011,2,434218.17,471580.34,-37362.17
2,2011,3,573838.05,434218.17,139619.88
3,2011,4,421527.77,573838.05,-152310.28
4,2011,5,650735.39,421527.77,229207.62
5,2011,6,641129.21,650735.39,-9606.18
6,2011,7,580714.73,641129.21,-60414.48
7,2011,8,612966.30,580714.73,32251.57
8,2011,9,924390.57,612966.30,311424.27
9,2011,10,965151.25,924390.57,40760.68


> There was a huge decrease in the total revenue in Dec 2011 compared to November 2011 since there were only 9 days of transactions.

In [18]:
# How did the monthly revenue compare to the average revenue in 2011?

query = """
SELECT
    year,
    month,
    total_revenue,
    ROUND(AVG(total_revenue) OVER (), 2) as avg_revenue,
    ROUND(total_revenue - AVG(total_revenue) OVER (), 2) as diff
FROM (
    SELECT
        CAST(DATE_PART(YEAR, invoice_date) as int) as year,
        CAST(DATE_PART(MONTH, invoice_date) as int) as month,
        ROUND(SUM(total_order_value), 2) as total_revenue
    FROM bootcamp.online_transactions_cleaned
    GROUP BY year, month) rev_table
WHERE year = 2011
ORDER BY month;
"""

pd.read_sql(query, connect)

,year,month,total_revenue,avg_revenue,diff
0,2011,1,471580.34,644585.29,-173004.95
1,2011,2,434218.17,644585.29,-210367.12
2,2011,3,573838.05,644585.29,-70747.24
3,2011,4,421527.77,644585.29,-223057.52
4,2011,5,650735.39,644585.29,6150.10
5,2011,6,641129.21,644585.29,-3456.08
6,2011,7,580714.73,644585.29,-63870.56
7,2011,8,612966.30,644585.29,-31618.99
8,2011,9,924390.57,644585.29,279805.28
9,2011,10,965151.25,644585.29,320565.96


In [19]:
# Which month had the biggest revenue?

query = """
SELECT
    CAST(DATE_PART(YEAR, invoice_date) as int) as year,
    CAST(DATE_PART(MONTH, invoice_date) as int) as month,
    SUM(total_order_value) as total_revenue
FROM bootcamp.online_transactions_cleaned
GROUP BY year, month
ORDER BY total_revenue DESC
LIMIT 1
"""
pd.read_sql(query, connect)

,year,month,total_revenue
0,2011,11,1118870.46


In [20]:
# How much was the monthly average order value?
# Divide the total sales value by the number of transactions completed during the period

query = """
SELECT
    CAST(DATE_PART(YEAR, invoice_date) as int) as year,
    CAST(DATE_PART(MONTH, invoice_date) as int) as month,
    ROUND(SUM(total_order_value)/COUNT(DISTINCT invoice), 2) as average_order_value
FROM bootcamp.online_transactions_cleaned
GROUP BY year, month
ORDER BY year, month
"""

pd.read_sql(query, connect)

,year,month,average_order_value
0,2010,12,324.14
1,2011,1,384.96
2,2011,2,367.67
3,2011,3,361.36
4,2011,4,310.40
5,2011,5,359.92
6,2011,6,380.27
7,2011,7,373.45
8,2011,8,407.02
9,2011,9,453.58


In [21]:
# How many orders were placed per country?

query = """
SELECT
    country,
    COUNT(DISTINCT invoice) as num_invoices
FROM bootcamp.online_transactions_cleaned
GROUP BY country
ORDER BY num_invoices DESC
"""

pd.read_sql(query, connect)

,country,num_invoices
0,United Kingdom,19583
1,Germany,578
2,France,439
3,EIRE,313
4,Belgium,117
5,Spain,100
6,Netherlands,97
7,Australia,67
8,Switzerland,65
9,Portugal,55


In [22]:
# How much were the total revenues per country?

query = """
SELECT
    country,
    ROUND(SUM(total_order_value), 2) as total_revenue
FROM bootcamp.online_transactions_cleaned
GROUP BY country
ORDER BY total_revenue DESC
"""

pd.read_sql(query, connect)

,country,total_revenue
0,United Kingdom,6815375.09
1,Netherlands,283479.54
2,EIRE,251557.47
3,Germany,200619.66
4,France,181571.54
5,Australia,136922.50
6,Switzerland,51859.40
7,Spain,51746.65
8,Belgium,36662.96
9,Japan,35419.79


In [23]:
# From which country did we generate the most and least revenue?

query = """
SELECT
    country,
    total_revenue,
    CASE WHEN rank_asc = 1 THEN 'least_performing' ELSE 'top_performing' END AS performance
FROM (
    SELECT 
        country,
        ROUND(SUM(total_order_value), 2) as total_revenue,
        DENSE_RANK() OVER (ORDER BY SUM(total_order_value)) as rank_asc,
        DENSE_RANK() OVER (ORDER BY SUM(total_order_value) DESC) as rank_desc
    FROM bootcamp.online_transactions_cleaned
    GROUP BY country) as ranking
WHERE 1 in (rank_asc, rank_desc)
"""

pd.read_sql(query, connect)

,country,total_revenue,performance
0,United Kingdom,6815375.09,top_performing
1,Saudi Arabia,131.17,least_performing


In [24]:
# What is the most popular and least popular product?

query = """
SELECT
    stock_code,
    description,
    total_qty_sold,
    CASE WHEN rank_asc = 1 THEN 'least_popular' ELSE 'most_popular' END AS popularity
FROM (
    SELECT 
        stock_code,
        description,
        SUM(quantity) as total_qty_sold,
        DENSE_RANK() OVER (ORDER BY SUM(quantity)) as rank_asc,
        DENSE_RANK() OVER (ORDER BY SUM(quantity) DESC) as rank_desc
    FROM bootcamp.online_transactions_cleaned
    GROUP BY stock_code, description) as ranking
WHERE 1 in (rank_asc, rank_desc)
"""

pd.read_sql(query, connect)

,stock_code,description,total_qty_sold,popularity
0,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,53119,most_popular
1,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-1475,least_popular


> Note: The least popular product has a total quantity sold of -1475 which could indicate returns or cancellation

In [25]:
# Preview data where quantity < 0 

query = """
SELECT *
FROM bootcamp.online_transactions_cleaned
WHERE quantity < 0
ORDER BY quantity 
LIMIT 10;
"""

pd.read_sql(query, connect)

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",2.08,-80995,-168469.60,2011-12-09 09:27:00,u16446,United Kingdom
1,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,1.04,-74215,-77183.60,2011-01-18 10:17:00,u12346,United Kingdom
2,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,0.03,-9360,-280.80,2010-12-02 14:23:00,u15838,United Kingdom
3,C550456,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,2.10,-3114,-6539.40,2011-04-18 13:08:00,u15749,United Kingdom
4,C550456,21175,GIN AND TONIC DIET METAL SIGN,1.85,-2000,-3700.00,2011-04-18 13:08:00,u15749,United Kingdom
5,C550456,85123A,CREAM HANGING HEART T-LIGHT HOLDER,2.55,-1930,-4921.50,2011-04-18 13:08:00,u15749,United Kingdom
6,C556522,22920,HERB MARKER BASIL,0.55,-1515,-833.25,2011-06-13 11:21:00,u16938,United Kingdom
7,C550456,47566B,TEA TIME PARTY BUNTING,2.55,-1300,-3315.00,2011-04-18 13:08:00,u15749,United Kingdom
8,C570556,20971,PINK BLUE FELT CRAFT TRINKET BOX,1.06,-1296,-1373.76,2011-10-11 11:10:00,u16029,United Kingdom
9,C569552,15034,PAPER POCKET TRAVELING FAN,0.07,-1200,-84.00,2011-10-04 17:44:00,u14533,United Kingdom


In [26]:
# How many invoices have negative quantities?

query = """
SELECT COUNT(DISTINCT invoice)
FROM bootcamp.online_transactions_cleaned
WHERE quantity < 0
"""

pd.read_sql(query, connect)

,count
0,3383


> Note: Out of 21,791 invoices, 3,383 have negative quantities

In [27]:
# What are the top 10 most returned items

query = """
SELECT description, SUM(quantity) as total_returned
FROM bootcamp.online_transactions_cleaned
WHERE quantity < 0
GROUP BY description
ORDER BY total_returned 
LIMIT 10
"""

pd.read_sql(query, connect)

,description,total_returned
0,"PAPER CRAFT , LITTLE BIRDIE",-80995
1,MEDIUM CERAMIC TOP STORAGE JAR,-74494
2,ROTATING SILVER ANGELS T-LIGHT HLDR,-9367
3,FAIRY CAKE FLANNEL ASSORTED COLOUR,-3150
4,CREAM HANGING HEART T-LIGHT HOLDER,-2578
5,GIN AND TONIC DIET METAL SIGN,-2030
6,HERB MARKER BASIL,-1527
7,FELTCRAFT DOLL MOLLY,-1447
8,TEA TIME PARTY BUNTING,-1424
9,PAPER POCKET TRAVELING FAN,-1385


In [28]:
# What is the number of stocks with the description 'Unknown'?

query = """
SELECT COUNT(DISTINCT stock_code)
FROM bootcamp.online_transactions_cleaned
WHERE description = 'Unknown'
"""

pd.read_sql(query, connect)

,count
0,18


In [29]:
# Preview stock_codes that have 'Unknown' description

query = """
SELECT DISTINCT stock_code, description
FROM bootcamp.online_transactions_cleaned
WHERE description = 'Unknown'
ORDER BY stock_code
"""

pd.read_sql(query, connect)

,stock_code,description
0,16151A,Unknown
1,16162M,Unknown
2,18007,Unknown
3,20964,Unknown
4,21703,Unknown
5,21704,Unknown
6,21705,Unknown
7,22686,Unknown
8,22878,Unknown
9,22889,Unknown


## Data Preprocessing
- Assess the data quality and structure and clean or transform it as necessary before performing further analysis, visualization, and segmentation of customers.

In [30]:
online_trans.head()

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,1.04,-74215,-77183.6,2011-01-18 10:17:00,u12346,United Kingdom
1,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,1.04,74215,77183.6,2011-01-18 10:01:00,u12346,United Kingdom
2,581180,23497,CLASSIC CHROME BICYCLE BELL,1.45,12,17.4,2011-12-07 15:52:00,u12347,Iceland
3,581180,21265,PINK GOOSE FEATHER TREE 60CM,1.95,12,23.4,2011-12-07 15:52:00,u12347,Iceland
4,537626,22773,GREEN DRAWER KNOB ACRYLIC EDWARDIAN,1.25,12,15.0,2010-12-07 14:57:00,u12347,Iceland


In [31]:
online_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399841 entries, 0 to 399840
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   invoice            399841 non-null  object        
 1   stock_code         399841 non-null  object        
 2   description        399841 non-null  object        
 3   price              399841 non-null  float64       
 4   quantity           399841 non-null  int64         
 5   total_order_value  399841 non-null  float64       
 6   invoice_date       399841 non-null  datetime64[ns]
 7   customer_id        399841 non-null  object        
 8   country            399841 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 27.5+ MB


In [32]:
online_trans.describe()

,price,quantity,total_order_value
count,399841.000000,399841.000000,399841.000000
mean,2.952914,12.224359,20.716904
std,7.317593,250.788860,425.471765
min,0.000000,-80995.000000,-168469.600000
25%,1.250000,2.000000,4.250000
50%,1.950000,5.000000,11.580000
75%,3.750000,12.000000,19.500000
max,1599.260000,80995.000000,168469.600000


> **Observation:**
> 1. There are outliers in the price column and there are records where the price is 0.00. These need investigation.
> 2. There are records that have negative values and matching positive values which needs investigation

In [33]:
online_trans.describe(include = 'object')

,invoice,stock_code,description,customer_id,country
count,399841,399841,399841,399841,399841
unique,21791,3679,3644,4363,37
top,576339,85123A,CREAM HANGING HEART T-LIGHT HOLDER,u17841,United Kingdom
freq,542,2065,2065,7800,356158


> **Observation:**
> 1. Note that stock_code has 3679 unique values and description has 3644 unique values
> 2. This means that there are 35 (3679-3644) stock_codes that share the same description with other stock_codes

#### Task 1 - Investigate stock codes that share the same description with other stock codes

In [34]:
# Number of unique stock_codes that have 'Unknown' description

online_trans[online_trans['description'] == 'Unknown']['stock_code'].nunique()

18

In [35]:
# list of descriptions that belong to 2 or more stock_codes

desc = online_trans.groupby(['description'])['stock_code'].nunique().to_frame()
desc.reset_index(inplace = True)
shared_desc = desc[desc['stock_code'] != 1]
shared_desc

,description,stock_code
788,COLOURING PENCILS BROWN TUBE,2
790,COLUMBIAN CANDLE RECTANGLE,2
791,COLUMBIAN CANDLE ROUND,2
1022,EAU DE NILE JEWELLED PHOTOFRAME,2
1247,FRENCH FLORAL CUSHION COVER,2
1250,FRENCH LATTICE CUSHION COVER,2
1268,FROSTED WHITE BASE,2
1893,"METAL SIGN,CUPCAKE SINGLE HOOK",3
2309,PINK FAIRY CAKE CUSHION COVER,2
2314,PINK FLOCK GLASS CANDLEHOLDER,2


In [36]:
# Convert the shared description into a list

shared_desc_list = list(shared_desc['description'])
shared_desc_list

['COLOURING PENCILS BROWN TUBE',
 'COLUMBIAN CANDLE RECTANGLE',
 'COLUMBIAN CANDLE ROUND',
 'EAU DE NILE JEWELLED PHOTOFRAME',
 'FRENCH FLORAL CUSHION COVER ',
 'FRENCH LATTICE CUSHION COVER ',
 'FROSTED WHITE BASE ',
 'METAL SIGN,CUPCAKE SINGLE HOOK',
 'PINK FAIRY CAKE CUSHION COVER',
 'PINK FLOCK GLASS CANDLEHOLDER',
 'PINK FLOWERS RABBIT EASTER',
 "RETRO PLASTIC 70'S TRAY",
 'RETRO PLASTIC DAISY TRAY',
 'RETRO PLASTIC POLKA TRAY',
 'ROUND BLUE CLOCK WITH SUCKER',
 'SQUARE CHERRY BLOSSOM CABINET',
 'Unknown',
 'WHITE BAMBOO RIBS LAMPSHADE']

In [37]:
def find_stock_w_shared_desc(df):
    """This function lists the stock codes that share the same description with another stock codes"""
    shared_desc = shared_desc_list
    stock_code_w_shared_desc = []
    for desc in shared_desc:
        stockcode = df[df['description'] == desc]['stock_code'].unique()
        stock_code_w_shared_desc.append(stockcode)
    return stock_code_w_shared_desc

# Apply the function to our dataframe
find_stock_w_shared_desc(online_trans)

[array(['10135', '10133'], dtype=object),
 array(['72131', '72133'], dtype=object),
 array(['72130', '72128'], dtype=object),
 array(['85023B', '85024B'], dtype=object),
 array(['20837', '20840'], dtype=object),
 array(['20838', '20835'], dtype=object),
 array(['79403', '79406'], dtype=object),
 array(['82613C', '82613B', '82613A'], dtype=object),
 array(['47585A', '47586A'], dtype=object),
 array(['79329', '79331'], dtype=object),
 array(['35910B', '35909B'], dtype=object),
 array(['79190A', '79192A'], dtype=object),
 array(['79190D', '79191D'], dtype=object),
 array(['79190B', '79191B'], dtype=object),
 array(['81952B', '81953B'], dtype=object),
 array(['84665', '84666'], dtype=object),
 array(['21704', '21703', '46000U', '21705', '46000S', '46000M', '18007',
        '23702', '22686', '46000R', '23444', '22889', '22878', '46000P',
        '16151A', '84247K', '16162M', '20964'], dtype=object),
 array(['40001', '40003'], dtype=object)]

In [38]:
# Verify stock codes 10135 and 10133

online_trans[online_trans['stock_code'].isin(['10135', '10133'])]

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
6656,560991,10135,COLOURING PENCILS BROWN TUBE,1.25,10,12.50,2011-07-22 13:29:00,u12438,Norway
11206,579529,10135,COLOURING PENCILS BROWN TUBE,1.25,10,12.50,2011-11-30 08:50:00,u12488,France
20292,544933,10135,COLOURING PENCILS BROWN TUBE,1.06,200,212.00,2011-02-25 08:44:00,u12633,Germany
24654,563613,10133,COLOURING PENCILS BROWN TUBE,0.42,20,8.40,2011-08-18 08:49:00,u12688,Israel
29017,537155,10133,COLOURING PENCILS BROWN TUBE,0.85,3,2.55,2010-12-05 13:05:00,u12748,United Kingdom
...,...,...,...,...,...,...,...,...,...
396864,562965,10133,COLOURING PENCILS BROWN TUBE,0.42,20,8.40,2011-08-11 10:45:00,u18226,United Kingdom
396918,540259,10135,COLOURING PENCILS BROWN TUBE,0.42,40,16.80,2011-01-06 09:00:00,u18226,United Kingdom
397961,539477,10133,COLOURING PENCILS BROWN TUBE,0.85,10,8.50,2010-12-19 14:58:00,u18245,United Kingdom
398708,559174,10133,COLOURING PENCILS BROWN TUBE,0.42,20,8.40,2011-07-07 10:04:00,u18263,United Kingdom


> **Observation:**
> 1. This is just to note that there are stock codes that share the same description with another stock code and there seems to be no issue about it.

#### Task 2 - Check for missing values and duplicate records

In [39]:
# Check for missing values

online_trans.isnull().sum()

invoice              0
stock_code           0
description          0
price                0
quantity             0
total_order_value    0
invoice_date         0
customer_id          0
country              0
dtype: int64

In [40]:
# Check for duplicate records

online_trans.duplicated().sum()

0

#### Task 3 - Investigate invoice numbers

In [41]:
# Investigate invoice numbers

online_trans['invoice'].str[0].value_counts()

5    391334
C      8507
Name: invoice, dtype: int64

In [42]:
# Preview records where invoice number starts with C

online_trans[online_trans['invoice'].str[0] == 'C']

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,1.04,-74215,-77183.60,2011-01-18 10:17:00,u12346,United Kingdom
312,C547388,22701,PINK DOG BOWL,2.95,-6,-17.70,2011-03-22 16:07:00,u12352,Norway
316,C547388,84050,PINK HEART SHAPE EGG FRYING PAN,1.65,-12,-19.80,2011-03-22 16:07:00,u12352,Norway
321,C547388,21914,BLUE HARMONICA IN BOX,1.25,-12,-15.00,2011-03-22 16:07:00,u12352,Norway
332,C547388,22413,METAL SIGN TAKE IT OR LEAVE IT,2.95,-6,-17.70,2011-03-22 16:07:00,u12352,Norway
...,...,...,...,...,...,...,...,...,...
398987,C577832,22423,REGENCY CAKESTAND 3 TIER,12.75,-1,-12.75,2011-11-22 10:18:00,u18274,United Kingdom
398990,C577386,23401,RUSTIC MIRROR WITH LACE HEART,6.25,-1,-6.25,2011-11-18 16:54:00,u18276,United Kingdom
398997,C577390,23401,RUSTIC MIRROR WITH LACE HEART,6.25,-1,-6.25,2011-11-18 17:01:00,u18276,United Kingdom
399006,C542086,22423,REGENCY CAKESTAND 3 TIER,12.75,-1,-12.75,2011-01-25 12:34:00,u18277,United Kingdom


> **Observation:**
> 1. There are 8507 invoices that start with C which indicates cancellation.

#### Task 4 - Investigate stock codes

In [43]:
# Investigate stock_codes

online_trans['stock_code'].str[0].value_counts()

2    330871
8     44096
4      8597
1      5304
7      5214
3      3590
9      1482
5       436
C       134
6        97
D        16
P         4
Name: stock_code, dtype: int64

> **Observation:**
> 1. There are 134, 16, and 4 records where stock_code start with C, D, and P respectively

In [44]:
# Check the stock_codes that start with C, D, and P

online_trans[online_trans['stock_code'].str[0].isin(['C', 'D', 'P'])]['stock_code'].unique()

array(['C2', 'PADS', 'DOT'], dtype=object)

> **Observation:**
> 1. There are stock_codes that are non-numeric: 'C2', 'DOT', 'PADS'

In [45]:
# Investigate records where stock_code is 'C2', 'DOT', and 'PADS'

alpha_stock_code = ['C2', 'DOT', 'PADS']
online_trans[online_trans['stock_code'].isin(alpha_stock_code)]

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
11521,554826,C2,CARRIAGE,18.000,2,36.000,2011-05-26 14:53:00,u12493,France
22616,539688,C2,CARRIAGE,150.000,1,150.000,2010-12-21 11:00:00,u12678,France
33406,577009,C2,CARRIAGE,50.000,1,50.000,2011-11-17 12:05:00,u12749,United Kingdom
68939,558908,C2,CARRIAGE,50.000,1,50.000,2011-07-05 09:06:00,u13338,United Kingdom
69202,556883,C2,CARRIAGE,50.000,1,50.000,2011-06-15 12:24:00,u13343,United Kingdom
...,...,...,...,...,...,...,...,...,...
263403,568200,PADS,PADS TO MATCH ALL CUSHIONS,0.001,1,0.001,2011-09-25 14:58:00,u16198,United Kingdom
267054,556969,C2,CARRIAGE,25.000,1,25.000,2011-06-16 10:11:00,u16257,United Kingdom
267385,563911,C2,CARRIAGE,50.000,1,50.000,2011-08-21 12:39:00,u16265,United Kingdom
267426,547571,C2,CARRIAGE,50.000,1,50.000,2011-03-24 09:52:00,u16265,United Kingdom


> **Observation/Assumption:**
> 1. There are 154 records with stock_code C2 (carriage), DOT(dotcom postage), and PADS(pads to match all cushions) which don't seem to be sale of products. These will be removed.

#### Data Quality Issue 1 - Records where stock code is C2, DOT, and PADS will be removed

In [46]:
# Delete records where stock_code is C2, DOT, and PADS

online_trans = online_trans[~online_trans['stock_code'].isin(alpha_stock_code)]

#### Task 5 - Investigate price with 0.00 values

In [47]:
# Investigate price with $0.00 values

online_trans[online_trans['price'] == 0]

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
3913,554037,22619,SET OF 6 SOLDIER SKITTLES,0.0,80,0.0,2011-05-20 14:13:00,u12415,Australia
4074,574138,23234,BISCUIT TIN VINTAGE CHRISTMAS,0.0,216,0.0,2011-11-03 11:26:00,u12415,Australia
5533,574469,22385,JUMBO BAG SPACEBOY DESIGN,0.0,12,0.0,2011-11-04 11:55:00,u12431,Australia
6764,577314,23407,SET OF 2 TRAYS HOME SWEET HOME,0.0,2,0.0,2011-11-18 13:23:00,u12444,Norway
7861,550188,22636,CHILDS BREAKFAST SET CIRCUS PARADE,0.0,1,0.0,2011-04-14 18:57:00,u12457,Switzerland
12050,561669,22960,JAM MAKING SET WITH JARS,0.0,11,0.0,2011-07-28 17:09:00,u12507,Spain
21240,537197,22841,ROUND CAKE TIN VINTAGE GREEN,0.0,1,0.0,2010-12-05 14:02:00,u12647,Germany
46071,574879,22625,RED KITCHEN SCALES,0.0,2,0.0,2011-11-07 13:22:00,u13014,United Kingdom
49756,540372,22090,PAPER BUNTING RETROSPOT,0.0,24,0.0,2011-01-06 16:41:00,u13081,United Kingdom
50238,540372,22553,PLASTERS IN TIN SKULLS,0.0,24,0.0,2011-01-06 16:41:00,u13081,United Kingdom


In [48]:
len(online_trans[online_trans['price'] == 0])

33

> **Observations/Assumptions:**
> 1. A total of 33 records have a price of 0.00, which could be related to inventory adjustment, samples or free items given to customers
> 2. If the price is 0.00 then there is no sales transaction or no sales revenue recorded
> 3. For the purpose of analysis, these transactions should be excluded as these are not considered sales transactions

#### Data Quality Issue 2 - Records where price is 0.00 will be removed

In [49]:
# Drop records where price = 0.00

online_trans = online_trans[online_trans['price'] != 0]

In [50]:
online_trans.describe()

,price,quantity,total_order_value
count,399654.000000,399654.000000,399654.000000
mean,2.907695,12.194974,20.680201
std,4.451996,250.062575,425.533339
min,0.030000,-80995.000000,-168469.600000
25%,1.250000,2.000000,4.250000
50%,1.950000,5.000000,11.580000
75%,3.750000,12.000000,19.500000
max,649.500000,80995.000000,168469.600000


> **Observations:**
> 1. For  price, looks like 649.50 is an outlier as it significantly differs from the rest of the other prices
> 2. For quantity, it shows that min/max values -80,995 and 80,995 extremely and significantly differ from the rest of the other quantity values. This needs further investigation.

#### Task 6 - Investigate outliers for price 

In [51]:
# Investigate records where price > 300

online_trans[online_trans['price'] > 300]

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
197106,556446,22502,PICNIC BASKET WICKER SMALL,649.5,1,649.5,2011-06-10 15:33:00,u15098,United Kingdom
197107,556444,22502,PICNIC BASKET WICKER SMALL,649.5,60,38970.0,2011-06-10 15:28:00,u15098,United Kingdom


> **Observation/Assumption:**
> 1. Note that stock_code 22502 with description PICNIC BASKET WICKER SMALL is the most expensive item priced at 649.50. There seems to be no issue about it.

#### Task 7 - Investigate price levels

In [52]:
# Investigate price levels

num_unique_price = online_trans.groupby(['stock_code'])['price'].nunique()
num_unique_price = num_unique_price.reset_index()
num_unique_price[num_unique_price['price'] > 1]

,stock_code,price
1,10080,2
6,10125,2
7,10133,2
8,10135,5
9,11001,3
...,...,...
3667,90214P,2
3668,90214R,2
3669,90214S,2
3672,90214V,2


> **Observation/Assumption:**
> 1. There are 2633 stock codes that have more than 1 price levels

In [53]:
# Determine how many price levels 

num_unique_price['price'].value_counts()

2     1263
1     1043
3      901
4      294
5       92
6       44
7       24
8       10
10       2
9        2
12       1
Name: price, dtype: int64

> It appears that only 1043 stock codes have 1 price level and the other stock codes have 2 to 12 different price levels

In [54]:
# Investigate stock code with 12 different price levels

num_unique_price[num_unique_price['price'] == 12]

,stock_code,price
393,21166,12


> Stock code 21166 has 12 different price levels

In [55]:
# List different price levels for stock code 21166

sorted(list(online_trans[online_trans['stock_code'] == '21166']['price'].unique()))

[1.4, 1.49, 1.53, 1.66, 1.69, 1.85, 1.95, 2.08, 2.21, 2.39, 2.4, 2.41]

> stock code 21166 have 12 different price levels ranging from 1.4 to 2.41

In [56]:
# Show records where stcok code is 21166

online_trans[online_trans['stock_code'] == '21166'].sort_values('price')

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
389407,569257,21166,COOK WITH WINE METAL SIGN,1.40,48,67.20,2011-10-03 11:24:00,u18102,United Kingdom
389176,569255,21166,COOK WITH WINE METAL SIGN,1.49,96,143.04,2011-10-03 11:01:00,u18102,United Kingdom
254054,544477,21166,COOK WITH WINE METAL SIGN,1.53,48,73.44,2011-02-21 10:07:00,u16029,United Kingdom
253917,552786,21166,COOK WITH WINE METAL SIGN,1.53,144,220.32,2011-05-11 12:14:00,u16029,United Kingdom
253925,558193,21166,COOK WITH WINE METAL SIGN,1.53,48,73.44,2011-06-27 12:25:00,u16029,United Kingdom
...,...,...,...,...,...,...,...,...,...
341750,567458,21166,COOK WITH WINE METAL SIGN,2.41,336,809.76,2011-09-20 12:21:00,u1745,United Kingdom
341807,574293,21166,COOK WITH WINE METAL SIGN,2.41,48,115.68,2011-11-03 15:32:00,u1745,United Kingdom
341632,572545,21166,COOK WITH WINE METAL SIGN,2.41,144,347.04,2011-10-24 16:48:00,u1745,United Kingdom
341771,567384,21166,COOK WITH WINE METAL SIGN,2.41,48,115.68,2011-09-20 10:23:00,u1745,United Kingdom


> **Observation/Assumption:**
> 1. Different price levels could indicate that there may be wholesale prices and retail prices or just simply price increase but since we don't have information about the store's pricing policy - these are just assumptions and for this reason, it is not necessary to make any changes

#### Task 8 - Investigate negative quantities

In [57]:
# Preview records with positive/negative quantities

online_trans[online_trans['quantity'].abs() >= 5000]

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,1.04,-74215,-77183.6,2011-01-18 10:17:00,u12346,United Kingdom
1,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,1.04,74215,77183.6,2011-01-18 10:01:00,u12346,United Kingdom
242293,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,0.03,-9360,-280.8,2010-12-02 14:23:00,u15838,United Kingdom
277641,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",2.08,-80995,-168469.6,2011-12-09 09:27:00,u16446,United Kingdom
277643,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",2.08,80995,168469.6,2011-12-09 09:15:00,u16446,United Kingdom


> **Observations/Assumptions:**
> 1. It shows that there are 3 extremely large quantities of sales and and 2 of them have corresponding cancellations.
> 2. Invoice 581483 with quantity value of 80995 was also cancelled the same day under invoice C581484. 
> 3. It's the same for invoice 541431 with quantity value of 74215, cancelled under invoice C541433.
> 4. For the purpose of further analysis, both the original and cancelled invoices should be removed since the negative quantity will offset the positive quantity. Hence, the net effect is zero.
> 5. However, cancelled invoice C536757 dated 2010-12-02 with quantity value of -9360 does not have its corresponding sale or order. Since our dataset covers only from 2010-12-01 to 2011-12-09, the corresponding sale for this cancellation could have been made prior to 2010-12-01 which is outside the coverage of the dataset. This record and all other cancelled invoices without a corresponding original invoice or order, ideally, should be removed or excluded in the analysis.

In [58]:
# Find the cancellation pairs: original invoice and corresponding cancelled invoice

online_trans.groupby([online_trans.customer_id, online_trans.stock_code, online_trans.quantity.abs()])\
            .filter(lambda x: (len(x.quantity.abs()) % 2 == 0) and (x.quantity.sum() == 0))\
            .sort_values(by=['customer_id', 'stock_code'])

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,1.04,-74215,-77183.6,2011-01-18 10:17:00,u12346,United Kingdom
1,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,1.04,74215,77183.6,2011-01-18 10:01:00,u12346,United Kingdom
845,543370,22839,3 TIER CAKE TIN GREEN AND CREAM,14.95,2,29.9,2011-02-07 14:51:00,u12359,Cyprus
857,C549955,22839,3 TIER CAKE TIN GREEN AND CREAM,14.95,-2,-29.9,2011-04-13 13:38:00,u12359,Cyprus
1199,573173,22941,CHRISTMAS LIGHTS 10 REINDEER,8.50,2,17.0,2011-10-28 10:10:00,u12362,Belgium
...,...,...,...,...,...,...,...,...,...
398978,575485,23245,SET OF 3 REGENCY CAKE TINS,4.95,4,19.8,2011-11-09 17:03:00,u18274,United Kingdom
398984,C577832,84509A,SET OF 4 ENGLISH ROSE PLACEMATS,3.75,-4,-15.0,2011-11-22 10:18:00,u18274,United Kingdom
398985,575485,84509A,SET OF 4 ENGLISH ROSE PLACEMATS,3.75,4,15.0,2011-11-09 17:03:00,u18274,United Kingdom
398972,C577832,84988,SET OF 72 PINK HEART PAPER DOILIES,1.45,-12,-17.4,2011-11-22 10:18:00,u18274,United Kingdom


> **Observations/Assumptions:**
> 1. There are 2015 pairs of original invoice and its corresponding cancelled invoice totalling to 4030 records. 
> 2. These are for 1:1 match - meaning for each customer, there are pairs of original invoice and cancelled invoice that have the same stock_code and matching positive/negative quantities. 
> 3. These records will be removed.

#### Data Quality Issue 3 - Records where for each customer, where there are pairs of original invoice and cancelled invoice that have the same stock_code and matching positive/negative quantities will be removed

In [59]:
# Drop records with pairs of original invoice and matching cancelled invoice

idx = online_trans.groupby([online_trans.customer_id, online_trans.stock_code, online_trans.quantity.abs()])\
            .filter(lambda x: (len(x.quantity.abs()) % 2 == 0) and (x.quantity.sum() == 0)).index

online_trans = online_trans.drop(idx)

In [60]:
online_trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395624 entries, 2 to 399840
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   invoice            395624 non-null  object        
 1   stock_code         395624 non-null  object        
 2   description        395624 non-null  object        
 3   price              395624 non-null  float64       
 4   quantity           395624 non-null  int64         
 5   total_order_value  395624 non-null  float64       
 6   invoice_date       395624 non-null  datetime64[ns]
 7   customer_id        395624 non-null  object        
 8   country            395624 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 30.2+ MB


In [61]:
online_trans.describe()

,price,quantity,total_order_value
count,395624.000000,395624.000000,395624.000000
mean,2.876935,12.319197,20.890201
std,4.163080,45.966884,95.688187
min,0.030000,-9360.000000,-6539.400000
25%,1.250000,2.000000,4.250000
50%,1.950000,5.000000,11.700000
75%,3.750000,12.000000,19.500000
max,649.500000,4800.000000,38970.000000


> **Observations/Assumptions:**
> 1. After deleting the cancellation pairs, there are still negative values in quantity - needs further investigation

In [62]:
# check for customers whose orders have negative quantities

cust_with_neg = list(online_trans[online_trans['quantity'] < 0]['customer_id'].unique())
print(cust_with_neg)
print('_' * 70)
print(f'There are {len(cust_with_neg)} customers whose orders have negative quantities')

['u12352', 'u12359', 'u12362', 'u12375', 'u12379', 'u1238', 'u12381', 'u12383', 'u12384', 'u12395', 'u12406', 'u12408', 'u12409', 'u1241', 'u12413', 'u12415', 'u12417', 'u12427', 'u12428', 'u12431', 'u12434', 'u12456', 'u12457', 'u12462', 'u12463', 'u12465', 'u12468', 'u12471', 'u12472', 'u12473', 'u12474', 'u12476', 'u12477', 'u12479', 'u12481', 'u12483', 'u12484', 'u12494', 'u125', 'u12501', 'u12517', 'u12523', 'u12528', 'u12536', 'u1254', 'u12544', 'u12553', 'u1256', 'u12565', 'u12566', 'u12567', 'u12569', 'u12572', 'u12576', 'u12577', 'u12578', 'u12583', 'u12584', 'u12585', 'u12586', 'u1259', 'u12597', 'u12598', 'u12601', 'u12605', 'u1261', 'u12612', 'u12613', 'u12616', 'u12619', 'u12621', 'u12625', 'u12626', 'u12628', 'u12633', 'u12637', 'u12647', 'u12649', 'u12652', 'u12656', 'u12657', 'u12665', 'u12666', 'u12668', 'u1267', 'u12674', 'u12678', 'u12679', 'u12681', 'u12683', 'u12693', 'u12697', 'u12703', 'u12705', 'u12708', 'u12709', 'u1271', 'u12712', 'u12719', 'u1272', 'u12721', 

> **Observation:**
> 1. There are 1308 customers whose orders have negative quantities

In [63]:
def check_cancellations(df):
    """This function definition is to capture records of a customer that have:
    1. partial cancellations
    2. cancellations only (without original orders)
    3. original orders, full cancellations, and re-orders
    and will require to pass a subset of a dataframe filtered to a specific customer that has negative values in orders
    """
    for i in df['quantity'].unique():
        
        #checking if for each negative quantity there is the same positive quantity
        if i < 0 and abs(i) in df['quantity'].unique():     
            print('There are matching positive quantities for', i)
            
            #checking stockcodes that have matching positive and negative quantity
            stockcode = list(df[df['quantity'] == i]['stock_code'].unique())
            print('Stockcodes that have matching positive and negative quantities:', stockcode)
        
            #subsetting the possible stockcodes and pair of positive + negative quantity
            subset_stockcode = df[df['stock_code'].isin(stockcode)]
            subset_pairs = subset_stockcode[(subset_stockcode['quantity'] == i)|(subset_stockcode['quantity'] == abs(i))]
        
            #iterating over each stockcode to group a pair
            for k in subset_pairs['stock_code'].unique():
                print('-' * 100)
                print(subset_pairs[subset_pairs['stock_code'] == k])
                print('-' * 100)
                print('-' * 100)
                
        # checking for negative quantities that have no matching positive quantities
        elif i < 0 and abs(i) not in df['quantity'].unique():
            print ('There are cancellations in these quantities:', i)
        
            #checking stockcodes that have negative quantity
            stockcode = list(df[df['quantity'] == i]['stock_code'].unique())
            print('stock_codes that have negative quantity:', stockcode)
        
            #subsetting the stockcodes that have negative quantity and cancelled order
            subset_stockcode = df[df['stock_code'].isin(stockcode)]
            cancelled_order = subset_stockcode[subset_stockcode['quantity'] == i]
        
            print('-' * 100)
            print(subset_stockcode)
            print('-' * 100)
            print(cancelled_order)
            print('-' * 100)
            print('-' * 100)

In [64]:
# Investigate customer u12352

u12352 = online_trans[online_trans['customer_id'] == 'u12352']
u12352

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
299,568699,21731,RED TOADSTOOL LED NIGHT LIGHT,1.65,12,19.80,2011-09-28 14:58:00,u12352,Norway
300,567505,37495,FAIRY CAKE BIRTHDAY CANDLE SET,3.75,4,15.00,2011-09-20 14:34:00,u12352,Norway
301,546869,84050,PINK HEART SHAPE EGG FRYING PAN,1.65,12,19.80,2011-03-17 16:00:00,u12352,Norway
302,544156,22423,REGENCY CAKESTAND 3 TIER,12.75,3,38.25,2011-02-16 12:33:00,u12352,Norway
303,567505,22980,PANTRY SCRUBBING BRUSH,1.65,12,19.80,2011-09-20 14:34:00,u12352,Norway
...,...,...,...,...,...,...,...,...,...
378,568699,21755,LOVE BUILDING BLOCK WORD,6.25,3,18.75,2011-09-28 14:58:00,u12352,Norway
379,546869,22701,PINK DOG BOWL,2.95,6,17.70,2011-03-17 16:00:00,u12352,Norway
380,C547388,22645,CERAMIC HEART FAIRY CAKE MONEY BANK,1.45,-12,-17.40,2011-03-22 16:07:00,u12352,Norway
381,567505,22692,DOORMAT WELCOME TO OUR HOME,8.25,2,16.50,2011-09-20 14:34:00,u12352,Norway


In [65]:
# Check cancellation scenarios for customer u12352

check_cancellations(u12352)

There are matching positive quantities for -6
Stockcodes that have matching positive and negative quantities: ['22701', '22413']
----------------------------------------------------------------------------------------------------
     invoice stock_code                      description  price  quantity  \
310   546869      22413  METAL SIGN TAKE IT OR LEAVE IT    2.95         6   
314   547390      22413  METAL SIGN TAKE IT OR LEAVE IT    2.95         6   
332  C547388      22413  METAL SIGN TAKE IT OR LEAVE IT    2.95        -6   

     total_order_value        invoice_date customer_id country  
310               17.7 2011-03-17 16:00:00      u12352  Norway  
314               17.7 2011-03-22 16:08:00      u12352  Norway  
332              -17.7 2011-03-22 16:07:00      u12352  Norway  
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

> **Assumptions:**
> - For customer u12352, there are 7 cases where there is one cancelled order and multiple orders for the same stock_code and quantity. It is assumed that the order was placed, cancelled then placed the order again
> - Example stock_code 22784 for 3 pieces:
        - ordered under invoice 546869 on 2011-03-17 
        - cancelled under invoice C547388 on 2011-03-22
        - placed the order again under invoice 547390 on 2011-03-22
> - Ideally, the orignal order and the subsequent cancelled oder should be removed.

In [66]:
# Investigate customer u12415

u12415 = online_trans[online_trans['customer_id'] == 'u12415']
u12415

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
3773,563614,22687,DOORMAT CHRISTMAS VILLAGE,6.75,20,135.00,2011-08-18 08:51:00,u12415,Australia
3774,563614,22492,MINI PAINT SET VINTAGE,0.55,576,316.80,2011-08-18 08:51:00,u12415,Australia
3775,540267,22857,ASSORTED EASTER GIFT TAGS,0.72,144,103.68,2011-01-06 11:12:00,u12415,Australia
3776,540267,21915,RED HARMONICA IN BOX,1.06,240,254.40,2011-01-06 11:12:00,u12415,Australia
3777,563614,23167,SMALL CERAMIC TOP STORAGE JAR,0.83,48,39.84,2011-08-18 08:51:00,u12415,Australia
...,...,...,...,...,...,...,...,...,...
4536,559919,23294,SET OF 6 SNACK LOAF BAKING CASES,0.72,384,276.48,2011-07-13 15:30:00,u12415,Australia
4537,569650,23012,GLASS APOTHECARY BOTTLE PERFUME,3.45,12,41.40,2011-10-05 12:44:00,u12415,Australia
4539,569650,23392,SPACEBOY ROCKET LOLLY MAKERS,1.79,96,171.84,2011-10-05 12:44:00,u12415,Australia
4541,559919,23122,PARTY CHARMS 50 PIECES,0.72,144,103.68,2011-07-13 15:30:00,u12415,Australia


In [67]:
# Check cancellation scenarios for customer u12415

check_cancellations(u12415)

There are cancellations in these quantities: -86
stock_codes that have negative quantity: ['22969']
----------------------------------------------------------------------------------------------------
      invoice stock_code                   description  price  quantity  \
4061   569650      22969  HOMEMADE JAM SCENTED CANDLES   1.25       192   
4210   543989      22969  HOMEMADE JAM SCENTED CANDLES   1.25       480   
4213   545475      22969  HOMEMADE JAM SCENTED CANDLES   1.25       288   
4227  C545525      22969  HOMEMADE JAM SCENTED CANDLES   1.25       -86   
4533   556917      22969  HOMEMADE JAM SCENTED CANDLES   1.25        96   

      total_order_value        invoice_date customer_id    country  
4061              240.0 2011-10-05 12:44:00      u12415  Australia  
4210              600.0 2011-02-15 09:52:00      u12415  Australia  
4213              360.0 2011-03-03 10:59:00      u12415  Australia  
4227             -107.5 2011-03-03 13:11:00      u12415  Australia  
453

> **Assumptions:**
> - For customer u12415, there is a case of partial cancellation
> - stock_code 22969:
        - placed an order for 288 pieces under invoice 545475 on 2011-03-03 10:59
        - cancelled 86 pieces under invoice C545525 after 2 hours on 2011-03-03 13:11
> - Ideally, the net quantity of 202 (288-86) should only be considered, but for the purpose of analysis, records that have partial cancellations need not to be removed

In [68]:
# Investigate customer u17548

u17548 = online_trans[online_trans['customer_id'] == 'u17548']
u17548

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
347093,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,0.29,-24,-6.96,2010-12-01 10:24:00,u17548,United Kingdom
347094,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,0.29,-24,-6.96,2010-12-01 10:24:00,u17548,United Kingdom
347095,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,1.65,-12,-19.80,2010-12-01 10:24:00,u17548,United Kingdom
347096,C536391,22553,PLASTERS IN TIN SKULLS,1.65,-24,-39.60,2010-12-01 10:24:00,u17548,United Kingdom
347101,C536391,22557,PLASTERS IN TIN VINTAGE PAISLEY,1.65,-12,-19.80,2010-12-01 10:24:00,u17548,United Kingdom
347103,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,0.29,-24,-6.96,2010-12-01 10:24:00,u17548,United Kingdom
347104,C536391,21484,CHICK GREY HOT WATER BOTTLE,3.45,-12,-41.40,2010-12-01 10:24:00,u17548,United Kingdom


In [69]:
# Check cancellation scenarios for customer u17548

check_cancellations(u17548)

There are cancellations in these quantities: -24
stock_codes that have negative quantity: ['21983', '21984', '22553', '21980']
----------------------------------------------------------------------------------------------------
        invoice stock_code                        description  price  \
347093  C536391      21983   PACK OF 12 BLUE PAISLEY TISSUES    0.29   
347094  C536391      21984   PACK OF 12 PINK PAISLEY TISSUES    0.29   
347096  C536391      22553             PLASTERS IN TIN SKULLS   1.65   
347103  C536391      21980  PACK OF 12 RED RETROSPOT TISSUES    0.29   

        quantity  total_order_value        invoice_date customer_id  \
347093       -24              -6.96 2010-12-01 10:24:00      u17548   
347094       -24              -6.96 2010-12-01 10:24:00      u17548   
347096       -24             -39.60 2010-12-01 10:24:00      u17548   
347103       -24              -6.96 2010-12-01 10:24:00      u17548   

               country  
347093  United Kingdom  
34709

> **Assumption:**
> - For customer u17548, there are only cancelled orders without any matching orders
> - Ideally, these records of cancelled orders without a corresponding original order, should be removed

In [70]:
# Check records where quantity < 0

online_trans[online_trans['quantity'] < 0].sort_values(by=['customer_id', 'stock_code'])

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
321,C547388,21914,BLUE HARMONICA IN BOX,1.25,-12,-15.00,2011-03-22 16:07:00,u12352,Norway
332,C547388,22413,METAL SIGN TAKE IT OR LEAVE IT,2.95,-6,-17.70,2011-03-22 16:07:00,u12352,Norway
380,C547388,22645,CERAMIC HEART FAIRY CAKE MONEY BANK,1.45,-12,-17.40,2011-03-22 16:07:00,u12352,Norway
312,C547388,22701,PINK DOG BOWL,2.95,-6,-17.70,2011-03-22 16:07:00,u12352,Norway
374,C547388,22784,LANTERN CREAM GAZEBO,4.95,-3,-14.85,2011-03-22 16:07:00,u12352,Norway
...,...,...,...,...,...,...,...,...,...
398829,C552720,84817,DANISH ROSE DECORATIVE PLATE,2.10,-2,-4.20,2011-05-11 09:49:00,u18272,United Kingdom
398990,C577386,23401,RUSTIC MIRROR WITH LACE HEART,6.25,-1,-6.25,2011-11-18 16:54:00,u18276,United Kingdom
398997,C577390,23401,RUSTIC MIRROR WITH LACE HEART,6.25,-1,-6.25,2011-11-18 17:01:00,u18276,United Kingdom
399006,C542086,22423,REGENCY CAKESTAND 3 TIER,12.75,-1,-12.75,2011-01-25 12:34:00,u18277,United Kingdom


###  Note: 
- 1. The only way to find and check for cancellation scenarios is to do the search by customer.
- 2. There are 6491 records where quantity < 0 and these records belong to 1308 customers which means we will have to apply the check_cancellations() function to 1308 customers and analyze the cancellation cases
- 3. It is not proper to remove just the cancellations (records that have negative quantities) as this will result to overstatement of sales transactions. 
- 4. This is just to note that there are three cancellation scenarios in the data:
        - partial cancellations
        - cancellations without the corresponding original orders
        - original orders that were cancelled then placed the orders again

#### Task 8 - Additional columns such as date, transaction period, day of the week, and hour need to be created for the purposes of further analysis, visualization, and customer segmentation

In [71]:
# Extract date, period, day, hour and create individual columns

online_trans['date'] = pd.to_datetime(online_trans['invoice_date']).dt.date
online_trans['period'] = online_trans['invoice_date'].dt.to_period('M')
online_trans['dow'] = online_trans['invoice_date'].dt.day_name()
online_trans['hour'] = online_trans['invoice_date'].dt.hour

In [72]:
online_trans.head()

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country,date,period,dow,hour
2,581180,23497,CLASSIC CHROME BICYCLE BELL,1.45,12,17.4,2011-12-07 15:52:00,u12347,Iceland,2011-12-07,2011-12,Wednesday,15
3,581180,21265,PINK GOOSE FEATHER TREE 60CM,1.95,12,23.4,2011-12-07 15:52:00,u12347,Iceland,2011-12-07,2011-12,Wednesday,15
4,537626,22773,GREEN DRAWER KNOB ACRYLIC EDWARDIAN,1.25,12,15.0,2010-12-07 14:57:00,u12347,Iceland,2010-12-07,2010-12,Tuesday,14
5,537626,22771,CLEAR DRAWER KNOB ACRYLIC EDWARDIAN,1.25,12,15.0,2010-12-07 14:57:00,u12347,Iceland,2010-12-07,2010-12,Tuesday,14
6,573511,84558A,3D DOG PICTURE PLAYING CARDS,2.95,36,106.2,2011-10-31 12:25:00,u12347,Iceland,2011-10-31,2011-10,Monday,12


In [73]:
online_trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395624 entries, 2 to 399840
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   invoice            395624 non-null  object        
 1   stock_code         395624 non-null  object        
 2   description        395624 non-null  object        
 3   price              395624 non-null  float64       
 4   quantity           395624 non-null  int64         
 5   total_order_value  395624 non-null  float64       
 6   invoice_date       395624 non-null  datetime64[ns]
 7   customer_id        395624 non-null  object        
 8   country            395624 non-null  object        
 9   date               395624 non-null  object        
 10  period             395624 non-null  period[M]     
 11  dow                395624 non-null  object        
 12  hour               395624 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), 

#### Task 9 - Load the preprocessed dataframe to a.pkl file and save in local folder

In [74]:
# Load the preprocessed dataframe to a .pkl file in local folder

online_trans.to_pickle('../data/online_trans.pkl')

In [75]:
# Load the preprocessed online_trans dataframe to a csv file for dashboard creation purposes

online_trans.to_csv('../data/online_trans.csv', index=False)